# Losses

Losses in vschaos are implemented using the `vschaos.criterions.Criterion` object, a sub-class of the pytorch loss objects providing addditional features a dynamical naming, loss algebra, and tracking. Alternative losses can be used to defined evolved variational auto-encoding modules such as for example beta-VAE, Wasserstein auto-encoders, and many more. 

This object outputs two arguments : the overall loss, and a tuple containing the individual losses of the criterion, if it is composed of several sub-losses.


In [1]:
import torch
import vschaos.criterions as crit
import vschaos.distributions as dist

# crit.M
l2 = crit.MSE()

out_params = [dist.Normal(torch.zeros(64, 10).normal_(), torch.zeros(64, 10).normal_()),
              dist.Normal(torch.zeros(64, 10).normal_(), torch.zeros(64, 10).normal_())]
target = [torch.zeros(64, 10).normal_(), torch.zeros(64, 10).normal_()]

loss_out, losses = l2(out_params, target)

# every Criterion should implement the Criterion.get_named_losses(), that returns the named losses from
#   individual tuples
named_losses = l2.get_named_losses(losses)
print(named_losses)

{'mse_loss_0': tensor(19.9847), 'mse_loss_1': tensor(19.5847)}


The `Criterion` objects also support basic algebras (+, -, /, \*, \*\*). In that case, the individual losses returned by the second argument represent the individual values, unaffected by the loss intern calculus. Moreover, loss tracking can be achieved directly in the corresponding criterion using the write function. Intern time tracking is also done, allowing to plot index losses in the time domain.

In [2]:
l1 = crit.L1()

reconstruction_loss = 0.1*l1 + 0.5*l2**2
loss_out, losses = reconstruction_loss(out_params, target)
print(loss_out, reconstruction_loss.get_named_losses(losses))

# write losses in the loss history
reconstruction_loss.write("train", losses)
print(reconstruction_loss.loss_history)

tensor(195.9405) {'l1_loss_0': tensor(1.1102), 'l1_loss_1': tensor(1.1275), 'mse_loss_0': tensor(9.9923), 'mse_loss_1': tensor(9.7923)}
{'train': {'l1_loss_0': {'values': array([1.1102383], dtype=float32), 'time': [1.958661]}, 'l1_loss_1': {'values': array([1.127469], dtype=float32), 'time': [1.958661]}, 'mse_loss_0': {'values': array([9.992333], dtype=float32), 'time': [1.9588670000000001]}, 'mse_loss_1': {'values': array([9.792345], dtype=float32), 'time': [1.9588670000000001]}}}


The library also defines higher-order criterions, such as the `ELBO`criterion that parses the output of the VAE to corresponding losses. Reconstruction and regularization are by default respectively `LogDensity` and `KLD`, but can be changed when creating the ELBO object.

In [3]:
from vschaos import vaes
import torch
import vschaos.criterions as crit
import vschaos.distributions as dist

reconstruction_loss = 0.3*crit.L1() + crit.MSE() + 0.4*crit.LogDensity()
reconstruction_loss = reconstruction_loss
# here are some specific losses for spectrums (set transform to 'stft' if applied on raw signals)
spectral_loss = crit.SpectralLoss(losses=['spec_conv', 'log_isd', 'log_diff', 'l2_mag'],
                                  weights=[1.0, 1.0, 1.0, 1.0], transform=None)
regularization_loss = crit.MMD()

elbo_loss = crit.ELBO(reconstruction_loss=reconstruction_loss, regularization_loss=regularization_loss)

input_params = {'dim':512, 'dist':dist.Normal}
hidden_prarams = [{'dim':200, 'nlayers':3, 'nn_lin':'Sigmoid'}, {'dim':100, 'nlayers':3, 'nn_lin':'Sigmoid'}]
latent_params = [{'dim':16, 'dist':dist.Normal}, {'dim':8, 'dist':dist.Normal}]

vae = vaes.VanillaVAE(input_params, latent_params, hidden_params=hidden_prarams)

x = torch.zeros(64, 512).normal_()
out_vae = vae(x)
loss_out, losses_out = elbo_loss(model=vae, target=x, out=out_vae)
# get named losses return individual losses *without weighting*, in order to monitor them properly
print(elbo_loss.get_named_losses(losses_out))

{'l1_loss_0': 439.4884338378906, 'mse_loss_0': 591.8547973632812, 'log_density_0': 1430.23779296875, 'mmd_0': 90.74755859375, 'mmd_1': 41.56732177734375}
